In [4]:
import jax.numpy as jnp
import jax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import KeyFunctions as me
import ObjFunc as of
import scipy as sp
import CritPointMR as mr
import torch as pt
from time import time as tmr

In [16]:
def OPTAccel(MxN, DataSet, EOS):
    import torch as pt
    from time import time as tmr
    import KeyFunctions as me
    import ObjFunc as of
    import scipy as sp
    import CritPointMR as mr
    import jax.numpy as jnp
    import jax
    import numpy as np
    import pandas as pd
    
    jax.config.update("jax_enable_x64", True)
    [y, Tc, Pc, w, C, R, Vc, k, mxNames] = me.LookUpMix(MxN, DataSet, EOS)
    
    default_guess = (jnp.dot(y, Tc).item(), jnp.dot(y, Pc).item())
    default_bound = [(default_guess[0], default_guess[0]+100), (default_guess[1], default_guess[1]*2.5)]

    t_0 = tmr()
    df_minima = sp.optimize.differential_evolution(of.CostFunction, bounds = default_bound, args = (y, Tc, Pc, w, k, R, C, EOS), mutation = 0.95, atol = 1e-14, init = 'sobol')
    t_df = tmr()-t_0

    path = me.GetRootPath()
    path = path+'/3-RootFinding/TrainedModels/MX_'+str(MxN)+'.trh'
    trainednn = mr.CompNN(C).to(pt.device("cpu"))
    trainednn.load_state_dict(pt.load(path))
    trainednn.eval()

    ypred = trainednn(pt.tensor(y)).detach().numpy()
    PcN = np.dot(np.ones(C)*1/C, Pc)
    TcN = np.dot(np.ones(C)*1/C, Tc)
    ML_guess = np.multiply(ypred, np.array([TcN, PcN]))
    ML_bound = [(ML_guess[0]*.90, ML_guess[0]*1.1), (ML_guess[1]*0.90, ML_guess[1]*1.1)]
    display(ML_bound)
    
    t_0 = tmr()
    bt_x, bt_fval,_,_ = sp.optimize.brute(of.CostFunction, ranges = ML_bound,full_output=True, args = (y, Tc, Pc, w, k, R, C, EOS), finish=sp.optimize.fmin)
    display(bt_x, bt_fval)
    t_br = tmr()-t_0
    
    
    US_conv = df_minima.fun < 1
    S_conv = bt_fval < 1
    
    t_frac = (t_df-t_br)/t_df
    itr_frac = (df_minima.nfev-400)/df_minima.nfev
    
    
    return US_conv, S_conv, t_frac, itr_frac

In [6]:
import pandas as pd
DataSet = 'DimJiaLi Unique'
EOS = 'PR'
MxNmax = 23
df = pd.DataFrame(index = range(MxNmax), columns = ['Unseeded Convergence', 'Seeded Convergence', 'Iteration Reduction', 'Time Reduction'])
for MxN in range(MxNmax):
    US_conv, S_conv, t_frac, itr_frac = OPTAccel(MxN, DataSet, EOS)
    df.iloc[MxN, :] = [US_conv, S_conv, np.round(t_frac*100, 3), np.round(itr_frac*100, 3)]
    df.to_csv('CompAccel.csv')
    

KeyboardInterrupt: 

from time import time as t
t0= t()
[y, Tc, Pc, w, C, R, Vc, k, mxNames] = me.LookUpMix(MxN, DataSet, EOS)
TP_guess = [4.41044394e+02, 6.25409061e+06]
def ModCostFunc(T, P, y, Tc, Pc, w, k, R, C, EOS):
    TP = np.array([T[0], P])
    F = of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
    return F

P_max = TP_guess[1]
P_min = TP_guess[1]*0.90
T_max = sp.optimize.brute(ModCostFunc, [(418.9921740807772, 463.0966134577012)], args = (P_max, y, Tc, Pc, w, k, R, C, EOS), Ns= 5)
T_min =  sp.optimize.brute(ModCostFunc, [(418.9921740807772, 463.0966134577012)], args = (P_min, y, Tc, Pc, w, k, R, C, EOS), Ns= 5)

def ModCostFunc(TP, y, Tc, Pc, w, k, R, C, EOS):
    TP[0] = (P_max-P_min)/(T_max-T_min)*(TP[0]-T_max)+P_max
    F = of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
    return F

bt_x, bt_fval,_,_ = sp.optimize.brute(ModCostFunc, ranges = [(418.9921740807772, 463.0966134577012),
 (5941386.081873446, 6566795.143123283)], full_output=True, args = (y, Tc, Pc, w, k, R, C, EOS), finish=sp.optimize.fmin, Ns= 5)
t_1 =t()-t0
display(t_1)
display(bt_fval)

In [18]:
t_frac

0.7192280359213763